``pip install flask``


``curl -X POST http://localhost:5000/image -H "Content-Type: application/json" -d '{"x": 3, "y": 4, "r": 255, "g": 100, "b": 50}'``


In [7]:
import requests
import numpy as np
import os
from PIL import Image
from scripts.wled_uploader import send_image_to_wled

# Configuration
WLED_IP = "led-matrix.local"
IMAGE_PATH = "./server-backup.png"
MATRIX_WIDTH = 16
MATRIX_HEIGHT = 16

def load_image():
    if os.path.exists(IMAGE_PATH):
        with Image.open(IMAGE_PATH) as img:
            img = img.convert("RGB")
            return np.array(img, dtype=np.uint8)
    else:
        return np.zeros((MATRIX_HEIGHT, MATRIX_WIDTH, 3), dtype=np.uint8)

def save_image(img_array):
    img = Image.fromarray(img_array, "RGB")
    img.save(IMAGE_PATH)

def serpentine_index(x, y, width=MATRIX_WIDTH):
    return y * width + x if y % 2 == 0 else y * width + (width - 1 - x)

def send_pixel_to_wled(x, y, rgb, width=MATRIX_WIDTH, wled_ip=WLED_IP):
    index = serpentine_index(x, y, width)
    hex_color = '{:02X}{:02X}{:02X}'.format(*rgb)
    payload = {
        "on": True,
        "seg": [
            {
                "id": 0,
                "i": [index, hex_color]
            }
        ]
    }
    try:
        url = f"http://{wled_ip}/json/state"
        resp = requests.post(url, json=payload, timeout=2)
        resp.raise_for_status()
    except Exception as e:
        print(f"Failed to send pixel to WLED: {e}")

def sync_image_to_wled(wled_ip=WLED_IP):
    try:
        send_image_to_wled(IMAGE_PATH, wled_ip, MATRIX_WIDTH * MATRIX_HEIGHT)
        print("Image synced to WLED.")
    except Exception as e:
        print(f"Failed to sync image to WLED: {e}")

# Initialize server
sync_image_to_wled(WLED_IP)

Image synced to WLED.


In [8]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Add CORS headers to every response
@app.after_request
def add_cors_headers(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type,Authorization"
    response.headers["Access-Control-Allow-Methods"] = "GET,POST,OPTIONS"
    return response

@app.route("/image", methods=["GET"])
def get_image():
    image = load_image()
    return jsonify(image.tolist())

@app.route("/image", methods=["POST"])
def update_pixel():
    image = load_image()
    data = request.json
    try:
        x = int(data["x"])
        y = int(data["y"])
        r = int(data["r"])
        g = int(data["g"])
        b = int(data["b"])
    except (KeyError, ValueError, TypeError):
        return "Invalid payload", 400

    if not (0 <= x < 16 and 0 <= y < 16):
        return "x or y out of bounds", 400
    for val in (r, g, b):
        if not (0 <= val <= 255):
            return "RGB values must be 0-255", 400

    image[y, x] = [r, g, b]
    save_image(image)
    send_pixel_to_wled(x, y, [r, g, b])

    return jsonify({"status": "ok", "updated_pixel": {"x": x, "y": y, "r": r, "g": g, "b": b}})

@app.route("/sync", methods=["POST"])
def sync_image():
    try:
        sync_image_to_wled(WLED_IP)
        return jsonify({"status": "ok", "message": "Image synced to WLED"})
    except Exception as e:
        return jsonify({"status": "error", "error": str(e)}), 500


app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.51.2.238:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2025 00:44:10] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:44:15] "OPTIONS /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:44:15] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:44:18] "POST /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:10] "OPTIONS /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:11] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:13] "POST /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:42] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:43] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 00:45:57] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 01:08:19] "OPTIONS /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 01:08:20] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 01:08:22] "POST /image HTTP/1.1" 200 -
127.0.0